# **To calibrate**


Keeping only valid tacks and gybes - stable TWA and BSP (std dev < 5 and  <3),  (TWA before - after)<5 (to be adjusted depending on TWS)

and BSP before and after statistically equal (t test for independent sampling (95%)) 

Define and save averages for before and after manouevers and mark roundings - dealing with circular data

*Noticed some delay issues when plotting for HDG and TWD or TWA.*

In [379]:
#Import data
import pandas as pd
from datetime import datetime
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import circmean

alpha = 0.05

#path of a boat_log
csv_path = r"C:\Users\matsa\OneDrive\Υπολογιστής\SailGP\SGP Data challenge VS\SGP_Data_Challenge\Data\Boat_logs\data_AUS.csv"

df = pd.read_csv(csv_path)
df["DATETIME"] = pd.to_datetime(df["DATETIME"])
df=df.sort_values(by='DATETIME', ascending= True)


**TWS Bins**

In [380]:
bin_width = 4
min_tws = int(df['TWS_SGP_km_h_1'].min())
max_tws = int(df['TWS_SGP_km_h_1'].max())+3

# Create bins with range from min to max, every 2 kmh
df['TWS_bins']=pd.cut(df['TWS_SGP_km_h_1'], bins=range(min_tws, max_tws + bin_width, bin_width))

# Functions

In [381]:
#calculates the tack angle based.
# returns the calculated tack angle based on whether it is an upwind tack or a downwind tack.
def calculate_tack_angle(before, after, istack):
    tack_angle = round(((after - before + 360) % 360) / 2,1)
    
    # Ensure smallest angle based on tack or gybe
    if istack == True:  # Upwind 
        return min(tack_angle, 180 - tack_angle) 
    else:  # Down
        return max(tack_angle, 180 - tack_angle)
    
    
# Color the 'A' column with a light blue background
def highlight_col(col):
    return ['background-color: lightblue' for _ in col]
    

# Define tack index points

In [382]:
# Find the indices where TWA changes sign
sign_changes = (df["TWA_SGP_deg"] * df["TWA_SGP_deg"].shift(-1) < 0).values

# Calculate time differences between consecutive rows - gaps between races ==> trouble.
df["time_diff"] = df["DATETIME"].diff().dt.total_seconds()

# Calculate the sign changes and store the result in a new column 'tackindexpoints' - allowing 5sec missing
df["tackjibes"] = (df["TWA_SGP_deg"].shift(-1).multiply(df["TWA_SGP_deg"], fill_value=0) < 0)&(df["time_diff"].shift(-1) <= 5)

df["absTWA"] = df["TWA_SGP_deg"].abs()
df["absAWA"] = df["AWA_SGP_deg"].abs()
df['tack'] = np.where(df['TWA_SGP_deg'] < 0, 'P', 'S') # port or starboard

In [383]:
alltackjibes = df[df["tackjibes"] == True].copy()
#alltackjibes contains only the rows where there is tack or gybe

# Find out the valid tackjibes points and check for validity

In [384]:
alpha = 0.05
df['datetime'] = df['DATETIME'].copy()
df = df.set_index('datetime')

# Create an empty list to store the tacks and jibes after testing
tackjibes = []

istack_value = (df["absTWA"] < 90) & (df["absTWA"].shift(-1) < 90) #if Tack --> True

# Define the variables for averaging
variables = {
    "TWS_SGP_km_h_1": "TWS",
    "BOAT_SPEED_km_h_1": "BSP",
    "GPS_SOG_km_h_1": "SOG",
    "absAWA": "AWA"
}

circvariables360 = {"TWD_SGP_deg": "TWD","HEADING_deg": "HDG", "GPS_COG_deg": "COG"}

circvariables180 = {"absTWA": "TWA"}

# Merge two dictionaries
merged_vars = {**variables, **circvariables360, **circvariables180}

In [385]:
# Loop through each detected manouever to find the time before and after and calculate the 10s averages 
for index in alltackjibes.index:
    index += 1 
       
    # Skip if TWS < 20 km/h or BSP < 10 km/h
    if df.iloc[index]["TWS_SGP_km_h_1"] <= 20 or df.iloc[index]["BOAT_SPEED_km_h_1"] <= 10:
        tackjibes.append({
            "datetime": df.index[index],
            "TWS_bins": df['TWS_bins'].iloc[index],
            "istack": istack_value.iloc[index],
            "from":df["tack"].iloc[index-1],
            "to":df["tack"].iloc[index+1],
            "isvalidcalib": False,
            "timebefore": 35,
            "timeafter": 35,
            "avgtime": 10,
            "isdeleted": False,
            "errcode": 1,
            "errmsg": "TWS less than 20kmh or BSP less than 10kmh"
        })
        continue  # Skip to next iteration
    
    max_p_value = 0
    max_p_value_range = None
    timestamp = df.index[index]
    time_before, time_after, avg_time = 0, 0, 0
    
    # Find best matching time ranges for manouevers
    for j in range(max(0, index - 20), index):
        subset_before = df.iloc[j - 10:j][["BOAT_SPEED_km_h_1", "absTWA"]]

        for i in range(index + 5, min(index + 30, len(df) - 1)):
            subset_after = df.iloc[i:i + 10][["BOAT_SPEED_km_h_1", "absTWA"]]

            # Check std and mean conditions
            if (
                subset_before["BOAT_SPEED_km_h_1"].std() < 3
                and subset_after["BOAT_SPEED_km_h_1"].std() < 3
                and subset_after["absTWA"].std() < 5
                and subset_before["absTWA"].std() < 5
                and abs(subset_after["absTWA"].mean() - subset_before["absTWA"].mean()) < 5
            ):
                t_stat, p_value = ttest_ind(subset_before["BOAT_SPEED_km_h_1"], subset_after["BOAT_SPEED_km_h_1"], equal_var=True)

                if p_value > alpha and p_value > max_p_value:
                    max_p_value = p_value
                    max_p_value_range = (j, i)

    if max_p_value_range:
        # Calculate time differences
        time_before = (timestamp - df.index[max_p_value_range[0]]).total_seconds()
        time_after = (df.index[max_p_value_range[1] + 10] - timestamp).total_seconds()
        avg_time = 10

        # Compute 10-second averages for before/after
        start_time_before = df.index[max_p_value_range[0]]-pd.Timedelta(seconds=10)
        end_time_before = df.index[max_p_value_range[0]] 

        start_time_after = df.index[max_p_value_range[1]]
        end_time_after = start_time_after + pd.Timedelta(seconds=10)

        before_avg = df.loc[start_time_before:end_time_before, list(merged_vars.keys())].copy().mean()
        after_avg = after_avg = df.loc[start_time_after:end_time_after, list(merged_vars.keys())].copy().mean()
        
        before_avg_circ360 = df.loc[start_time_before:end_time_before, list(circvariables360.keys())] \
                        .apply(lambda x: circmean(x, high=360, low=0), axis=0)

        after_avg_circ360 = df.loc[start_time_after:end_time_after, list(circvariables360.keys())] \
                        .apply(lambda x: circmean(x, high=360, low=0), axis=0)
        
        before_avg_circ180 = df.loc[start_time_before:end_time_before, list(circvariables180.keys())] \
                        .apply(lambda x: circmean(x, high=180, low=0), axis=0)

        after_avg_circ180 = df.loc[start_time_after:end_time_after, list(circvariables180.keys())] \
                        .apply(lambda x: circmean(x, high=180, low=0), axis=0)

        
        # Create new row to add the averages
        new_row = {
            "datetime": timestamp,
            "TWS_bins": df['TWS_bins'].iloc[index],
            "istack": istack_value.iloc[index],
            "from":df["tack"].iloc[index-1],
            "to":df["tack"].iloc[index+1],
            "isvalidcalib": True,
            "timebefore": time_before,
            "timeafter": time_after,
            "avgtime": avg_time,
            "errmsg": ""
            
        }

        # Add before/after P/S averages to the new row
        if df.iloc[index-1]["TWA_SGP_deg"]<0:
            for var, short_name in merged_vars.items():
                new_row[f"{short_name}_P"] = round(before_avg[var], 1)
                new_row[f"{short_name}_S"] = round(after_avg[var], 1)
            
        else:
            for var, short_name in merged_vars.items():
                new_row[f"{short_name}_P"] = round(after_avg[var], 1)
                new_row[f"{short_name}_S"] = round(before_avg[var], 1)
        
        for var, short_name in merged_vars.items():
            new_row[f"{short_name}_delta"] = new_row[f"{short_name}_P"] - new_row[f"{short_name}_S"]
            
        for var, short_name in circvariables360.items():
            new_row[f"angle_{short_name}"] = calculate_tack_angle(before_avg_circ360[var], after_avg_circ360[var],istack_value.iloc[index])
        
        for var, short_name in circvariables180.items():
            new_row[f"angle_{short_name}"] = round((before_avg_circ180[var]+ after_avg_circ180[var])/2,1)
        
        tackjibes.append(new_row)
           
    else:
        # If no valid time range found, add a default row
        tackjibes.append({
            "datetime": timestamp,
            "TWS_bins": df['TWS_bins'].iloc[index],
            "istack": istack_value.iloc[index],
            "from":df["tack"].iloc[index-1],
            "to":df["tack"].iloc[index+1],
            "isvalidcalib": False,
            "timebefore": 35,
            "timeafter": 35,
            "avgtime": 10,
            "errmsg": ""
        })

# Convert the list of dictionaries to a DataFrame
tackjibes_df = pd.DataFrame(tackjibes)  # All tacks and jibes detected

In [386]:
valid_tackjibes_df=tackjibes_df[tackjibes_df["isvalidcalib"]].sort_values('istack', ascending=False)


In [387]:
new_columns_order = [
    'datetime', 'istack', 'from','to', 'isvalidcalib', 'timebefore',
    'timeafter', 'avgtime', 'TWS_bins','TWS_P', 'TWS_S','TWS_delta', 'TWD_P', 'TWD_S', "TWD_delta", 'BSP_P', 'BSP_S','BSP_delta','SOG_P', 'SOG_S', 'SOG_delta','HDG_P', 'HDG_S', 'COG_P', 'COG_S',
    'TWA_P', 'TWA_S', 'TWA_delta', 'AWA_P', 'AWA_S','AWA_delta', 'angle_TWA','angle_HDG', 'angle_COG']
valid_tackjibes_df=valid_tackjibes_df[new_columns_order]

In [388]:

valid_tackjibes_df['hdg-twa']=valid_tackjibes_df['angle_HDG']-valid_tackjibes_df['angle_TWA']
valid_tackjibes_df['cog-hdg']=valid_tackjibes_df['angle_COG']-valid_tackjibes_df['angle_HDG']

In [389]:
valid_tackjibes_df.columns

Index(['datetime', 'istack', 'from', 'to', 'isvalidcalib', 'timebefore',
       'timeafter', 'avgtime', 'TWS_bins', 'TWS_P', 'TWS_S', 'TWS_delta',
       'TWD_P', 'TWD_S', 'TWD_delta', 'BSP_P', 'BSP_S', 'BSP_delta', 'SOG_P',
       'SOG_S', 'SOG_delta', 'HDG_P', 'HDG_S', 'COG_P', 'COG_S', 'TWA_P',
       'TWA_S', 'TWA_delta', 'AWA_P', 'AWA_S', 'AWA_delta', 'angle_TWA',
       'angle_HDG', 'angle_COG', 'hdg-twa', 'cog-hdg'],
      dtype='object')

# Group by 'istack' and calculate the mean of the specified variables 

(TWS, TWD BSP SOG TWA AWA HDG COG port, starboard and delta, as well as turn angles based on twa hdg and cog)

In [390]:
columns_to_average = [
    'TWS_P', 'TWS_S','TWS_delta', 'TWD_P', 'TWD_S', "TWD_delta", 'BSP_P', 'BSP_S','BSP_delta','SOG_P', 'SOG_S', 'SOG_delta',
    'TWA_P', 'TWA_S', 'TWA_delta','AWA_P', 'AWA_S','AWA_delta','HDG_P', 'HDG_S', 'COG_P', 'COG_S','angle_TWA','angle_HDG', 'angle_COG','hdg-twa', 'cog-hdg' 
]

averages = valid_tackjibes_df.groupby('istack')[columns_to_average].mean().round(1)
averages = averages.sort_values(by='istack', ascending=False)

In [391]:
averages

,TWS_P,TWS_S,TWS_delta,TWD_P,TWD_S,TWD_delta,BSP_P,BSP_S,BSP_delta,SOG_P,...,AWA_delta,HDG_P,HDG_S,COG_P,COG_S,angle_TWA,angle_HDG,angle_COG,hdg-twa,cog-hdg
istack,,,,,,,,,,,,,,,,,,,,,
True,36.7,39.2,-2.5,62.9,67.2,-4.3,55.6,55.5,0.2,57.4,...,-0.2,111.4,19.8,112.4,19.7,44.6,45.8,46.3,1.2,0.6
False,37.3,38.2,-0.9,61.4,64.5,-3.1,78.8,78.9,-0.1,78.0,...,-1.3,213.7,275.2,212.2,275.0,152.0,149.2,148.6,-2.8,-0.6


# Split in TWS Bins

In [392]:
averages_byTWS = valid_tackjibes_df.sort_values(by='istack', ascending=False)


In [393]:
averages_byTWS.columns

Index(['datetime', 'istack', 'from', 'to', 'isvalidcalib', 'timebefore',
       'timeafter', 'avgtime', 'TWS_bins', 'TWS_P', 'TWS_S', 'TWS_delta',
       'TWD_P', 'TWD_S', 'TWD_delta', 'BSP_P', 'BSP_S', 'BSP_delta', 'SOG_P',
       'SOG_S', 'SOG_delta', 'HDG_P', 'HDG_S', 'COG_P', 'COG_S', 'TWA_P',
       'TWA_S', 'TWA_delta', 'AWA_P', 'AWA_S', 'AWA_delta', 'angle_TWA',
       'angle_HDG', 'angle_COG', 'hdg-twa', 'cog-hdg'],
      dtype='object')

In [394]:
columns_to_average = [
    'TWS_P', 'TWS_S','TWS_delta', 'TWD_P', 'TWD_S', "TWD_delta", 'BSP_P', 'BSP_S','BSP_delta','SOG_P', 'SOG_S', 'SOG_delta',
    'TWA_P', 'TWA_S', 'TWA_delta','AWA_P', 'AWA_S','AWA_delta','HDG_P', 'HDG_S', 'COG_P', 'COG_S','angle_TWA','angle_HDG', 'angle_COG','hdg-twa', 'cog-hdg'
]
averages_byTWS = averages_byTWS.groupby(by=['istack','TWS_bins'])[columns_to_average].mean().round(1)

In [395]:
averages_byTWS

TWS_P  TWS_S  TWS_delta  TWD_P  TWD_S  TWD_delta  BSP_P  \
istack TWS_bins                                                            
False  (34, 38]   36.6   37.5       -1.0   61.0   65.1       -4.2   78.5   
       (38, 42]   40.2   40.7       -0.5   63.1   61.9        1.2   80.2   
True   (34, 38]   34.2   37.9       -3.7   63.5   69.1       -5.5   55.1   
       (38, 42]   40.0   41.0       -0.9   62.0   64.7       -2.7   56.3   

                 BSP_S  BSP_delta  SOG_P  ...  AWA_delta  HDG_P  HDG_S  COG_P  \
istack TWS_bins                           ...                                   
False  (34, 38]   78.7       -0.2   77.7  ...       -1.2  212.1  275.7  210.8   
       (38, 42]   79.6        0.6   79.1  ...       -1.9  219.9  273.3  218.1   
True   (34, 38]   55.2       -0.1   56.8  ...       -0.8  112.4   20.2  113.0   
       (38, 42]   55.8        0.5   58.3  ...        0.6  110.0   19.3  111.6   

                 COG_S  angle_TWA  angle_HDG  angle_COG  hdg-twa  cog-hdg  
istack TWS_bins                                                            
False  (34, 38]  275.5      151.4      148.2      147.6     -3.2     -0.6  
       (38, 42]  273.0      154.3      153.3      152.5     -1.0     -0.8  
True   (34, 38]   19.8       44.1       46.0       46.6      2.0      0.5  
       (38, 42]   19.6       45.2       45.4       46.0      0.1      0.6  

[4 rows x 27 columns]

# Mark Roundings

In [396]:
# Assign 1 if leg_diff == 1, else 0
df["mark_rounding"] = (df["TRK_LEG_NUM_unk"].diff() == 1).astype(int)
df=df.reset_index()
mark_roundings=df.loc[df['mark_rounding']==1]

In [397]:
# Initialize list to store results
mark_roundings_df = []

# Dictionary for shorthand variable names
variables = {
    "BOAT_SPEED_km_h_1": "BSP",
    "TWS_SGP_km_h_1": "TWS"
}
circvariables360={"TWD_SGP_deg": "TWD"}
merged_vars_roundings = {**variables, **circvariables360}

# Iterate over indices
for index in mark_roundings.index:
    # Calculate means safely
    rounding_before = df.loc[max(0, index-6):max(0, index-1), list(variables.keys())].mean().round(1)
    rounding_after = df.loc[min(len(df)-1, index+1):min(len(df)-1, index+6), list(variables.keys())].mean().round(1)
    rounding_before360 = df.loc[max(0, index-6):max(0, index-1), list(circvariables360.keys())] \
                        .apply(lambda x: circmean(x, high=360, low=0), axis=0).round(1)

    rounding_after360 = df.loc[min(len(df)-1, index+1):min(len(df)-1, index+6), list(circvariables360.keys())] \
                        .apply(lambda x: circmean(x, high=360, low=0), axis=0).round(1)
        
    # Construct new row
    new_row = {
        "datetime": df['datetime'].iloc[index],
        "BOAT": df['BOAT'].iloc[0],
        "Race": df['TRK_RACE_NUM_unk'].iloc[index],
        "isrounding": df['mark_rounding'].iloc[index],
        "from": df["TRK_LEG_NUM_unk"].iloc[index-1] if index > 0 else None,
        "to": df["TRK_LEG_NUM_unk"].iloc[index+1] if index < len(df)-1 else None,
    }

    # Identify leg type and assign metrics accordingly
    leg_num = df.iloc[index-1]["TRK_LEG_NUM_unk"] if index > 0 else None

    if leg_num in {2, 4, 6}: #Down to Up
        for var, short_name in variables.items():
            new_row[f"{short_name}_Up"] = round(rounding_after[var], 1)
            new_row[f"{short_name}_Down"] = round(rounding_before[var], 1)
            new_row[f"{short_name}_delta"] = new_row[f"{short_name}_Up"] - new_row[f"{short_name}_Down"]
        for var, short_name in circvariables360.items():
            new_row[f"{short_name}_Up"] = round(rounding_after360[var], 1)
            new_row[f"{short_name}_Down"] = round(rounding_before360[var], 1)
            new_row[f"{short_name}_delta"] = new_row[f"{short_name}_Up"] - new_row[f"{short_name}_Down"]

    elif leg_num in {3, 5}: #Up to Down
        for var, short_name in variables.items():
            new_row[f"{short_name}_Up"] = round(rounding_before[var], 1)
            new_row[f"{short_name}_Down"] = round(rounding_after[var], 1)
            new_row[f"{short_name}_delta"] = new_row[f"{short_name}_Up"] - new_row[f"{short_name}_Down"]
        for var, short_name in circvariables360.items():
            new_row[f"{short_name}_Up"] = round(rounding_before360[var], 1)
            new_row[f"{short_name}_Down"] = round(rounding_after360[var], 1)
            new_row[f"{short_name}_delta"] = new_row[f"{short_name}_Up"] - new_row[f"{short_name}_Down"]
    
    # Append the result
    mark_roundings_df.append(new_row)

for var, short_name in merged_vars_roundings.items():
            new_row[f"{short_name}_delta"] = new_row[f"{short_name}_Up"] - new_row[f"{short_name}_Down"]
# Convert list to DataFrame
mark_roundings_df = pd.DataFrame(mark_roundings_df).dropna()

In [398]:
#add average row at the end
mean_row = mark_roundings_df.mean(numeric_only=True)
mean_row.name = "Average"
mark_roundings_df = pd.concat([mark_roundings_df, mean_row.to_frame().T])

In [399]:

mark_roundings_df=mark_roundings_df.style.apply(highlight_col, subset=['TWS_delta','TWD_delta'])

In [400]:
mark_roundings_df

,datetime,BOAT,Race,isrounding,from,to,BSP_Up,BSP_Down,BSP_delta,TWS_Up,TWS_Down,TWS_delta,TWD_Up,TWD_Down,TWD_delta
2,2025-01-19 03:09:26,AUS,25011905.000000,1.000000,2.000000,3.000000,58.600000,78.300000,-19.700000,41.000000,39.900000,1.100000,54.800000,58.400000,-3.600000
3,2025-01-19 03:11:26,AUS,25011905.000000,1.000000,3.000000,4.000000,64.100000,76.600000,-12.500000,36.800000,36.600000,0.200000,59.800000,63.300000,-3.500000
4,2025-01-19 03:12:57,AUS,25011905.000000,1.000000,4.000000,5.000000,57.700000,71.800000,-14.100000,42.000000,41.000000,1.000000,68.100000,64.100000,4.000000
5,2025-01-19 03:15:01,AUS,25011905.000000,1.000000,5.000000,6.000000,57.700000,80.300000,-22.600000,37.900000,37.200000,0.700000,64.000000,61.400000,2.600000
6,2025-01-19 03:16:14,AUS,25011905.000000,1.000000,6.000000,7.000000,53.000000,76.600000,-23.600000,40.700000,39.800000,0.900000,58.600000,62.200000,-3.600000
9,2025-01-19 03:30:27,AUS,25011906.000000,1.000000,2.000000,3.000000,41.200000,75.100000,-33.900000,35.400000,34.400000,1.000000,62.200000,66.800000,-4.600000
10,2025-01-19 03:32:44,AUS,25011906.000000,1.000000,3.000000,4.000000,50.900000,78.100000,-27.200000,35.100000,34.300000,0.800000,62.100000,59.500000,2.600000
11,2025-01-19 03:33:59,AUS,25011906.000000,1.000000,4.000000,5.000000,56.100000,85.500000,-29.400000,39.600000,42.000000,-2.400000,56.200000,58.400000,-2.200000
12,2025-01-19 03:36:07,AUS,25011906.000000,1.000000,5.000000,6.000000,43.400000,57.600000,-14.200000,33.000000,34.400000,-1.400000,62.000000,61.800000,0.200000
13,2025-01-19 03:37:50,AUS,25011906.000000,1.000000,6.000000,7.000000,33.200000,21.200000,12.000000,39.500000,38.800000,0.700000,44.800000,53.100000,-8.300000


# Save file

In [401]:
# Save valid_tackjibes_df, averages, and mark_roundings_df in the same Excel file
with pd.ExcelWriter(f"Calibration_{df['BOAT'].iloc[0]}.xlsx", engine='xlsxwriter') as writer:
    valid_tackjibes_df.to_excel(writer, sheet_name="TackJibes", index=False)
    averages.to_excel(writer, sheet_name="Averages", index=True)
    averages_byTWS.to_excel(writer, sheet_name="Averages by TWS", index=True)
    mark_roundings_df.to_excel(writer, sheet_name="Mark Roundings", index=True)

    
print(f"The file is saved as Calibration_{df['BOAT'].iloc[0]}.xlsx" )

The file is saved as Calibration_AUS.xlsx
